In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
print("Loading Dataframe...")
alpha = time.time()

df_iawa = pd.read_excel("IAWA_Africa_modernOnly.xlsx", sheet_name="IAWA_Afrique_net_modern", header=2, na_filter=False)
df_powoAP = pd.read_excel("wcvp_taxon_distri_anatomy_net.xlsx", sheet_name="Acan-Plum", na_filter=False)
df_powoPZ = pd.read_excel("wcvp_taxon_distri_anatomy_net.xlsx", sheet_name="Poac-Zigo", na_filter=False)
df_dist1 = pd.read_excel("wcvp_taxon_distri_anatomy_net.xlsx", sheet_name="Distribution1_net", na_filter=False)
df_dist2 = pd.read_excel("wcvp_taxon_distri_anatomy_net.xlsx", sheet_name="Distribution2_net", na_filter=False)
df_cepam = pd.read_excel("CEPAM_feature_net.xlsx", sheet_name="CEPAM", header=2,na_filter=False)

print(f"Dataframes loading time : {time.time() - alpha} seconds")

Loading Dataframe...
Dataframes loading time : 549.3081862926483 seconds


In [44]:
iawa_label = ["iawa_family", "iawa_genus", "iawa_taxon", "accepted_id", "powo_family", "powo_genus", "powo_taxon", "powo_authors",
         "continent", "region", "area"]

errors_label = ["taxon", "iawa_line"]
errors = []
BUG = None

In [4]:
def extract_iawa_information(row):
    family = ""
    original_string = df_iawa["Taxa"][row].replace("?", "")
    #print(original_string)
    for x in original_string.split("|"):
        x = x.replace("Synonym:", "")
        family = re.search("[A-Z][A-Z]+\s[A-Z][A-Z]+|[A-Z][A-Z]+", x).group(0)
        genre = re.search(
            "[A-Z][a-z]+\s(spp\.|sp\.|SPP\.|SP\.)|[A-Z][a-z]+\s[a-z]+\s(subsp\.|cf\.|var\.)+\s[a-z]+|[A-Z][a-z]+\s[a-z\-]+|[A-Z][a-z]+", x).group(0)
        if re.search("\([A-Z][A-Z,\s]+\)", x):
            usual_name = re.search("\([A-Z][A-Z,\s]+\)", x).group(0)
        else:
            usual_name = ""
        return family, genre, usual_name, original_string

In [5]:
def get_id_by_taxon(taxon,df1,df2, org_str):
    global BUG
    matches = [" sp.", " spp.", " SPP.", " SP."," spp", " group", " subgrp"]
    taxon = taxon.replace(" cf", "")
    category = "taxon_name"
    if any([x in taxon for x in matches]):
        for x in matches:
            taxon = taxon.replace(x, "")
        category = "genus"
    
    test1 = df1[df1[category].values == taxon.strip()]['accepted_plant_name_id'].tolist()
    test2 = df2[df2[category].values == taxon.strip()]['accepted_plant_name_id'].tolist()
    
    if not test1 and not test2:
        errors.append([taxon, org_str])
        return None
        
    if not test1:
        if test2[0] == '':
            return df2[df2[category].values == taxon.strip()]['plant_name_id'].values[0]
        return test2[0]
    if test1[0] == '':
        return df1[df1[category].values == taxon.strip()]['plant_name_id'].values[0]
    return test1[0]

In [6]:
def get_plant_by_id(plant_id, df1,df2, plant):
    p1 = df1[df1["plant_name_id"].values == plant_id]
    p2 = df2[df2["plant_name_id"].values == plant_id]
    
    if p1.empty:
        if p2.empty:
            raise Exception(f"{plant}\nNo match id found for {plant_id} value type {type(plant_id)} test: {np.nan == plant_id}...")
        return p2['plant_name_id'].values[0], p2['family'].values[0], p2['genus'].values[0], p2['taxon_name'].values[0], p2['taxon_authors'].values[0]
    return p1['plant_name_id'].values[0], p1['family'].values[0], p1['genus'].values[0], p1['taxon_name'].values[0], p1['taxon_authors'].values[0]

In [7]:
def get_region_by_id(plant_id, df1, df2):
    r1 = df1[df1["plant_name_id"].values == plant_id]
    r2 = df2[df2["plant_name_id"].values == plant_id]
    
    if r1.empty:
        return r2["continent"].values, r2["region"].values, r2["area"].values
    return r1["continent"].values, r1["region"].values, r1["area"].values

In [49]:
# Execution
errors = []
extract_family = []
extract_genre = []
extract_usual_name = []
new_iawa = []

alpha = time.time()
print("Starting extraction")
iawa_label = ["iawa_family", "iawa_genus", "iawa_taxon", "accepted_id", "powo_family", "powo_genus", "powo_taxon", "powo_authors",
         "continent", "region", "area"]
iawa_label.extend(list(df_cepam.columns[4:]))
for row in df_iawa.index:
    family, genre, usual_name, original_string = extract_iawa_information(row)
    accepted_id = get_id_by_taxon(genre, df_powoAP, df_powoPZ, original_string)
    if not accepted_id:
        continue
    plant_id, accepted_family, accepted_genus, accepted_taxon, accepted_authors = get_plant_by_id(accepted_id, df_powoAP, df_powoPZ, original_string)
    continent, region, area = get_region_by_id(accepted_id, df_dist1, df_dist2)
    
    data = []
    for column in list(df_iawa.columns[1:]):
        data.append(df_iawa[column][row])
    for x,y,z in zip(continent, region, area):
        temp = [family, genre, usual_name,
                    plant_id, accepted_family, accepted_genus, accepted_taxon, accepted_authors,
                    x, y, z]
        temp.extend(data)
        new_iawa.append(temp)

print(f"Extraction done in {time.time() - alpha} seconds...")
pd.DataFrame(errors, columns=errors_label).to_csv("errors.csv", index=False, encoding="utf-8")
pd.DataFrame(new_iawa, columns=iawa_label).to_csv("generated_iawa.csv", index=False, encoding="utf-8", sep=";")

#print(to_extract)

Starting extraction
Extraction done in 372.6055817604065 seconds...


In [41]:
genus = re.search(
    "[A-Z][a-z]+\s(spp\.|sp\.)|[A-Z][a-z]+\s[a-z]+\s(subsp\.|cf\.|var\.)+\s[a-z]+|[A-Z][a-z]+\s[a-z\-]+|[A-Z][a-z]+",
          "LECYTHIDACEAE Oubanguia sp.").group(0)
print(genus)

Oubanguia sp.


In [16]:
df_cepam = pd.read_excel("CEPAM_feature_net.xlsx", sheet_name="CEPAM", header=2,na_filter=False)

In [28]:
experiment = list(df_cepam.columns)[4:]
for x in experiment:
    print(x)
    if len(x.split("-", 1)) > 1:
        number, name = x.split("-", 1)
    else:
        number, name = x.split(chr(8211),1)
    print(f"{number} : named {name}")
    

1 - Growth ring boundaries distinct
1  : named  Growth ring boundaries distinct
2 - Growth ring boundaries indistinct or absent
2  : named  Growth ring boundaries indistinct or absent
3 - Wood ring-porous
3  : named  Wood ring-porous
4 - Wood semi-ring-porous
4  : named  Wood semi-ring-porous
5 - Wood diffuse-porous
5  : named  Wood diffuse-porous
6 - Vessels in tangential bands
6  : named  Vessels in tangential bands
7 - Vessels in diagonal and / or radial pattern
7  : named  Vessels in diagonal and / or radial pattern
8 - Vessels in dendritic pattern
8  : named  Vessels in dendritic pattern
9 - Vessels exclusively solitary (90% or more)
9  : named  Vessels exclusively solitary (90% or more)
10 - Vessels in radial multiples of 4 or more common
10  : named  Vessels in radial multiples of 4 or more common
11 - Vessel clusters common
11  : named  Vessel clusters common
12 - Solitary vessel outline angular
12  : named  Solitary vessel outline angular
13 - Simple perforation plates
13  : n

In [27]:
"27 – Large ≥ 10 µm".split("-")
print(ord("-"))
print(ord("27 – Large ≥ 10 µm"[3]))

45
8211


In [32]:
list(df_iawa.columns[1:])

['1  Growth ring boundaries distinct',
 '2  Growth ring boundaries indistinct or absent',
 '3  Wood ringporous',
 '4  Wood semiringporous',
 '5  Wood diffuseporous',
 '6  Vessels in tangential bands',
 '7  Vessels in diagonal and / or radial pattern',
 '8  Vessels in dendritic pattern',
 '9  Vessels exclusively solitary (90% or more)',
 '10  Vessels in radial multiples of 4 or more common',
 '11  Vessel clusters common',
 '12  Solitary vessel outline angular',
 '13  Simple perforation plates',
 '14  Scalariform perforation plates',
 '15  Scalariform perforation plates with ≤ 10 bars',
 '16  Scalariform perforation plates with 10  20 bars',
 '17  Scalariform perforation plates with 20  40 bars',
 '18  Scalariform perforation plates with ≥ 40 bars',
 '19  Reticulate, foraminate, and / or other types of multiple perforation plates',
 '20  Intervessel pits scalariform',
 '21  Intervessel pits opposite',
 '22  Intervessel pits alternate',
 '23  Shape of alternate pits polygonal',
 '24  Minu

In [35]:
vector = []
for row in df_iawa.index:
    data = []
    for column in list(df_iawa.columns[1:]):
        data.append(df_iawa[column][row])
    vector.append(data)

['1  Growth ring boundaries distinct', '2  Growth ring boundaries indistinct or absent', '3  Wood ringporous', '4  Wood semiringporous', '5  Wood diffuseporous', '6  Vessels in tangential bands', '7  Vessels in diagonal and / or radial pattern', '8  Vessels in dendritic pattern', '9  Vessels exclusively solitary (90% or more)', '10  Vessels in radial multiples of 4 or more common', '11  Vessel clusters common', '12  Solitary vessel outline angular', '13  Simple perforation plates', '14  Scalariform perforation plates', '15  Scalariform perforation plates with ≤ 10 bars', '16  Scalariform perforation plates with 10  20 bars', '17  Scalariform perforation plates with 20  40 bars', '18  Scalariform perforation plates with ≥ 40 bars', '19  Reticulate, foraminate, and / or other types of multiple perforation plates', '20  Intervessel pits scalariform', '21  Intervessel pits opposite', '22  Intervessel pits alternate', '23  Shape of alternate pits polygonal', '24  Minute ≤ 4 µm', '25  Small 